In [ ]:
library(Seurat)
library(monocle)
library(ggplot2)
library(ggpubr)
library(dplyr)
library(ggsci)
library(data.table)
library(tidyverse)
library(psych)
library(qgraph)
library(igraph)
library(tidyverse)

# 总体数据

In [3]:
#导入注释好的seurat对象（已注释）
sce<-readRDS("/data_alluser/CXY/keti/singlecell_eqtl/result/RDS/all_subtype_annotated.rds")

In [4]:
####筛选部分细胞#####
sample_seob <- function(obj,group.by="Type",sp.size=NULL,diet="true",sp.total=1000) {
all <- obj
if (diet=="true") {
all <- DietSeurat(all)
}
if (is.null(sp.size)) {
nlen <- length(unique(all@meta.data[,group.by]))
sp.size <- ceiling(sp.total/nlen)
}
seob_list <- list()
i <- 1
for (sc in unique(all@meta.data[,group.by])){
cellist <- colnames(all)[which(all@meta.data[,group.by] == sc)]
ob <- subset(all, cells=cellist)
if (length(colnames(ob)) > sp.size) {
ob <- subset(ob,cells=sample(colnames(ob), sp.size))
}
seob_list[[i]] <- ob
i <- i+1
}
all <- Reduce(merge,seob_list)
return(all)
}

sce<- sample_seob(sce,group.by="Type",diet="true",sp.total=30000)

In [5]:
saveRDS(sce,'/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/sce.rds',compress = F)

In [6]:
cellphonedb <- readRDS('/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/sce.rds')

In [8]:
write.table(as.matrix(cellphonedb@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/input/cellphonedb_count.txt', sep='\t', quote=F)


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 4.4 GiB”


In [10]:
meta_data <- cbind(rownames(cellphonedb@meta.data), cellphonedb@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA



In [13]:
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/input/cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [2]:
count<-fread("/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/input/cellphonedb_count.txt")

Warning message in fread("/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/input/cellphonedb_count.txt"):
“Detected 25649 column names but the data has 25650 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


In [3]:
meta<-fread("/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/input/cellphonedb_meta.txt")

In [ ]:
cellphonedb method statistical_analysis  cellphonedb_meta.txt  cellphonedb_count.txt      --counts-data=gene_name

# hc_covid

## hc

In [4]:
cellphonedb <- readRDS('/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/sce.rds')

In [5]:
table(cellphonedb@meta.data$Status1)


COVID-19  Healthy 
   20189     5460 

In [6]:
Healthy<-subset(cellphonedb,subset=Status1=="Healthy")
Healthy@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025779_GATCTAGAGTTACGGG-1,GSM2025779,4005,1289,3.046192,Healthy,4230,1289,9,9,CD4 T,CD4 T memory,Healthy,Healthy10
GSM2025779_AGCGGTCTCGTGGACC-1,GSM2025779,1634,908,13.157895,Healthy,3753,1010,0,0,CD4 T,CD4 T memory,Healthy,Healthy10
GSM2025772_AGCGTATAGAGGGATA-1,GSM2025772,4950,1886,10.666667,Healthy,4647,1886,0,0,CD4 T,CD4 T memory,Healthy,Healthy3
GSM2025772_GGACGTCGTGATGCCC-1,GSM2025772,2113,724,3.691434,Healthy,4007,756,0,0,CD4 T,CD4 T memory,Healthy,Healthy3
GSM2025777_AGAGTGGGTGTGAAAT-1,GSM2025777,1157,626,14.952463,Healthy,3426,830,0,0,CD4 T,CD4 T memory,Healthy,Healthy8
GSM2025771_GCATGCGTCACTGGGC-1,GSM2025771,6450,2168,4.511628,Healthy,5007,2153,0,0,CD4 T,CD4 T memory,Healthy,Healthy2
GSM2025777_TGCTACCTCTCTGTCG-1,GSM2025777,4672,1461,8.989726,Healthy,4513,1461,0,0,CD4 T,CD4 T memory,Healthy,Healthy8
GSM2025775_CGTCACTCAGTCACTA-1,GSM2025775,5489,1644,4.572782,Healthy,4739,1644,0,0,CD4 T,CD4 T memory,Healthy,Healthy6
GSM2025778_TCACAAGAGAGACGAA-1,GSM2025778,4630,1461,6.436285,Healthy,4496,1461,0,0,CD4 T,CD4 T memory,Healthy,Healthy9


In [7]:
write.table(as.matrix(Healthy@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/covid_hc_input/hc/hc_status1_cellphonedb_count.txt', sep='\t', quote=F)


In [8]:
meta_data <- cbind(rownames(Healthy@meta.data), Healthy@meta.data[,'Status1', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/covid_hc_input/hc/hc_status1_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
cellphonedb method statistical_analysis  hc_status1_cellphonedb_meta.txt  hc_status1_cellphonedb_count.txt      --counts-data=gene_name

## COVID

In [13]:
COVID<-subset(cellphonedb,subset=Status1=="COVID-19")
table(COVID@meta.data$Status1)
COVID@meta.data


COVID-19 
   20189 

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025752_ACAGCTAAGAGGTTAT-1,GSM2025752,1858,885,11.948332,Severe,3859,951,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025766_TTGTAGGTCCTCGCAT-1,GSM2025766,1694,815,8.618654,Recover,3880,888,0,0,CD4 T,CD4 T memory,COVID-19,Recover9
GSM2025731_CTAAGACGTCCTCCAT-1,GSM2025731,4426,1213,5.173972,asymptomatic case,4422,1213,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case4
GSM2025738_CTACGTCTCGCATGGC-1,GSM2025738,5143,1577,3.616566,Moderate,4664,1577,0,0,CD4 T,CD4 T memory,COVID-19,Moderate6
GSM2025752_GTTCTCGCATCACGTA-1,GSM2025752,4775,1658,5.926702,Severe,4566,1658,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025760_CACACCTGTGCAGGTA-1,GSM2025760,3160,1214,8.544304,Recover,3983,1214,0,0,CD4 T,CD4 T memory,COVID-19,Recover3
GSM2025740_TGGGCGTTCCTCAACC-1,GSM2025740,2963,1196,14.579818,Moderate,3932,1198,0,0,CD4 T,CD4 T memory,COVID-19,Moderate8
GSM2025735_CTACCCATCAACACTG-1,GSM2025735,8154,2294,5.629139,Moderate,5036,2139,0,0,CD4 T,CD4 T memory,COVID-19,Moderate3
GSM2025752_TGACAACTCGCACTCT-1,GSM2025752,3780,1255,4.153439,Severe,4150,1255,0,0,CD4 T,CD4 T memory,COVID-19,Severe7


In [14]:
write.table(as.matrix(COVID@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/covid_hc_input/covid/covid_status1_cellphonedb_count.txt', sep='\t', quote=F)


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 3.5 GiB”


In [15]:
meta_data <- cbind(rownames(COVID@meta.data), COVID@meta.data[,'Status1', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/covid_hc_input/covid/covid_status1_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
cellphonedb method statistical_analysis  covid_status1_cellphonedb_meta.txt  covid_status1_cellphonedb_count.txt      --counts-data=gene_name

# COVID-19/healthy_type

In [51]:
table(cellphonedb@meta.data$Type)


       B    CD4 T    CD8 T      cDC Monocyte       NK      pDC Platelet 
    3750     3750     3750     2510     3750     3750     1251     3138 

In [52]:
cellphonedb@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025752_ACAGCTAAGAGGTTAT-1,GSM2025752,1858,885,11.948332,Severe,3859,951,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025779_GATCTAGAGTTACGGG-1,GSM2025779,4005,1289,3.046192,Healthy,4230,1289,9,9,CD4 T,CD4 T memory,Healthy,Healthy10
GSM2025779_AGCGGTCTCGTGGACC-1,GSM2025779,1634,908,13.157895,Healthy,3753,1010,0,0,CD4 T,CD4 T memory,Healthy,Healthy10
GSM2025766_TTGTAGGTCCTCGCAT-1,GSM2025766,1694,815,8.618654,Recover,3880,888,0,0,CD4 T,CD4 T memory,COVID-19,Recover9
GSM2025731_CTAAGACGTCCTCCAT-1,GSM2025731,4426,1213,5.173972,asymptomatic case,4422,1213,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case4
GSM2025738_CTACGTCTCGCATGGC-1,GSM2025738,5143,1577,3.616566,Moderate,4664,1577,0,0,CD4 T,CD4 T memory,COVID-19,Moderate6
GSM2025752_GTTCTCGCATCACGTA-1,GSM2025752,4775,1658,5.926702,Severe,4566,1658,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025760_CACACCTGTGCAGGTA-1,GSM2025760,3160,1214,8.544304,Recover,3983,1214,0,0,CD4 T,CD4 T memory,COVID-19,Recover3
GSM2025772_AGCGTATAGAGGGATA-1,GSM2025772,4950,1886,10.666667,Healthy,4647,1886,0,0,CD4 T,CD4 T memory,Healthy,Healthy3


## healthy

In [41]:
Healthy<-subset(cellphonedb,subset=Status1=="Healthy")
Healthy@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025779_GATCTAGAGTTACGGG-1,GSM2025779,4005,1289,3.046192,Healthy,4230,1289,9,9,CD4 T,CD4 T memory,Healthy,Healthy10
GSM2025779_AGCGGTCTCGTGGACC-1,GSM2025779,1634,908,13.157895,Healthy,3753,1010,0,0,CD4 T,CD4 T memory,Healthy,Healthy10
GSM2025772_AGCGTATAGAGGGATA-1,GSM2025772,4950,1886,10.666667,Healthy,4647,1886,0,0,CD4 T,CD4 T memory,Healthy,Healthy3
GSM2025772_GGACGTCGTGATGCCC-1,GSM2025772,2113,724,3.691434,Healthy,4007,756,0,0,CD4 T,CD4 T memory,Healthy,Healthy3
GSM2025777_AGAGTGGGTGTGAAAT-1,GSM2025777,1157,626,14.952463,Healthy,3426,830,0,0,CD4 T,CD4 T memory,Healthy,Healthy8
GSM2025771_GCATGCGTCACTGGGC-1,GSM2025771,6450,2168,4.511628,Healthy,5007,2153,0,0,CD4 T,CD4 T memory,Healthy,Healthy2
GSM2025777_TGCTACCTCTCTGTCG-1,GSM2025777,4672,1461,8.989726,Healthy,4513,1461,0,0,CD4 T,CD4 T memory,Healthy,Healthy8
GSM2025775_CGTCACTCAGTCACTA-1,GSM2025775,5489,1644,4.572782,Healthy,4739,1644,0,0,CD4 T,CD4 T memory,Healthy,Healthy6
GSM2025778_TCACAAGAGAGACGAA-1,GSM2025778,4630,1461,6.436285,Healthy,4496,1461,0,0,CD4 T,CD4 T memory,Healthy,Healthy9


In [42]:
table(Healthy@meta.data$Type)


       B    CD4 T    CD8 T      cDC Monocyte       NK      pDC Platelet 
     785      820      909      545      682      852      418      449 

In [47]:
write.table(as.matrix(COVID@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/covid_hc_input/covid/covid_cellphonedb_count.txt', sep='\t', quote=F)


In [48]:
meta_data <- cbind(rownames(Healthy@meta.data), Healthy@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/covid_hc_input/hc/hc_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

## covid

In [44]:
COVID<-subset(cellphonedb,subset=Status1=="COVID-19")
table(COVID@meta.data$Type)
COVID@meta.data


       B    CD4 T    CD8 T      cDC Monocyte       NK      pDC Platelet 
    2965     2930     2841     1965     3068     2898      833     2689 

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025752_ACAGCTAAGAGGTTAT-1,GSM2025752,1858,885,11.948332,Severe,3859,951,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025766_TTGTAGGTCCTCGCAT-1,GSM2025766,1694,815,8.618654,Recover,3880,888,0,0,CD4 T,CD4 T memory,COVID-19,Recover9
GSM2025731_CTAAGACGTCCTCCAT-1,GSM2025731,4426,1213,5.173972,asymptomatic case,4422,1213,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case4
GSM2025738_CTACGTCTCGCATGGC-1,GSM2025738,5143,1577,3.616566,Moderate,4664,1577,0,0,CD4 T,CD4 T memory,COVID-19,Moderate6
GSM2025752_GTTCTCGCATCACGTA-1,GSM2025752,4775,1658,5.926702,Severe,4566,1658,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025760_CACACCTGTGCAGGTA-1,GSM2025760,3160,1214,8.544304,Recover,3983,1214,0,0,CD4 T,CD4 T memory,COVID-19,Recover3
GSM2025740_TGGGCGTTCCTCAACC-1,GSM2025740,2963,1196,14.579818,Moderate,3932,1198,0,0,CD4 T,CD4 T memory,COVID-19,Moderate8
GSM2025735_CTACCCATCAACACTG-1,GSM2025735,8154,2294,5.629139,Moderate,5036,2139,0,0,CD4 T,CD4 T memory,COVID-19,Moderate3
GSM2025752_TGACAACTCGCACTCT-1,GSM2025752,3780,1255,4.153439,Severe,4150,1255,0,0,CD4 T,CD4 T memory,COVID-19,Severe7


In [49]:
write.table(as.matrix(COVID@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/covid_hc_input/covid/covid_cellphonedb_count.txt', sep='\t', quote=F)


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 3.5 GiB”


In [50]:
meta_data <- cbind(rownames(COVID@meta.data), COVID@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/covid_hc_input/covid/covid_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
cellphonedb method statistical_analysis  hc_cellphonedb_meta.txt  hc_cellphonedb_count.txt      --counts-data=gene_name

In [ ]:
cellphonedb method statistical_analysis  covid_cellphonedb_meta.txt  covid_cellphonedb_count.txt      --counts-data=gene_name

# 5状态-healthy

In [66]:
table(cellphonedb@meta.data$Status)


asymptomatic case           Healthy          Moderate           Recover 
             2737              5460              5765              6393 
           Severe 
             5294 

## asymptomatic

In [54]:
asymptomatic<-subset(cellphonedb,subset=Status=="asymptomatic case")
table(asymptomatic@meta.data$Type)
asymptomatic@meta.data


       B    CD4 T    CD8 T      cDC Monocyte       NK      pDC Platelet 
     364      345      351      302      297      480      187      411 

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025731_CTAAGACGTCCTCCAT-1,GSM2025731,4426,1213,5.173972,asymptomatic case,4422,1213,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case4
GSM2025731_CACACCTGTGTTCTTT-1,GSM2025731,514,307,1.750973,asymptomatic case,3335,707,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case4
GSM2025728_GCCAAATCATTGCGGC-1,GSM2025728,4249,1387,4.330431,asymptomatic case,4313,1387,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case1
GSM2025732_CTCTAATAGGGTATCG-1,GSM2025732,4469,1357,2.931305,asymptomatic case,4439,1357,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case5
GSM2025732_CCTCAGTAGATCGGGT-1,GSM2025732,4479,1250,4.264345,asymptomatic case,4437,1250,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case5
GSM2025730_GCGCGATGTGCAGACA-1,GSM2025730,2277,851,1.010101,asymptomatic case,3951,870,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case3
GSM2025731_GATTCAGAGGCTAGAC-1,GSM2025731,5509,1667,6.571066,asymptomatic case,4736,1667,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case4
GSM2025732_TCAGCAATCTGGTTCC-1,GSM2025732,4295,1260,4.447031,asymptomatic case,4341,1260,0,0,CD4 T,CD4 T memory,COVID-19,asymptomatic case5
GSM2025732_AGGTCCGCAAACCCAT-1,GSM2025732,3798,1401,4.976303,asymptomatic case,4133,1401,9,9,CD4 T,CD4 T memory,COVID-19,asymptomatic case5


In [55]:
write.table(as.matrix(asymptomatic@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/asymptomatic/asymptomatic_cellphonedb_count.txt', sep='\t', quote=F)


In [56]:
meta_data <- cbind(rownames(asymptomatic@meta.data), asymptomatic@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/asymptomatic/asymptomatic_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
cellphonedb method statistical_analysis  asymptomatic_cellphonedb_meta.txt  asymptomatic_cellphonedb_count.txt      --counts-data=gene_name

## Moderate           

In [57]:
Moderate<-subset(cellphonedb,subset=Status=="Moderate")
table(Moderate@meta.data$Type)
Moderate@meta.data


       B    CD4 T    CD8 T      cDC Monocyte       NK      pDC Platelet 
     997      875      860      399      718     1079      210      627 

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025738_CTACGTCTCGCATGGC-1,GSM2025738,5143,1577,3.616566,Moderate,4664,1577,0,0,CD4 T,CD4 T memory,COVID-19,Moderate6
GSM2025740_TGGGCGTTCCTCAACC-1,GSM2025740,2963,1196,14.579818,Moderate,3932,1198,0,0,CD4 T,CD4 T memory,COVID-19,Moderate8
GSM2025735_CTACCCATCAACACTG-1,GSM2025735,8154,2294,5.629139,Moderate,5036,2139,0,0,CD4 T,CD4 T memory,COVID-19,Moderate3
GSM2025739_ACGCCGAGTTCTCATT-1,GSM2025739,3028,1143,3.863937,Moderate,3964,1143,0,0,CD4 T,CD4 T memory,COVID-19,Moderate7
GSM2025734_ATCTACTAGGGCATGT-1,GSM2025734,4165,1779,5.930372,Moderate,4261,1779,0,0,CD4 T,CD4 T memory,COVID-19,Moderate2
GSM2025742_TGCTACCAGCGTTTAC-1,GSM2025742,4278,1437,9.116410,Moderate,4325,1437,0,0,CD4 T,CD4 T memory,COVID-19,Moderate10
GSM2025733_GTCACGGTCATGCTCC-1,GSM2025733,5935,1473,3.133951,Moderate,4734,1470,0,0,CD4 T,CD4 T memory,COVID-19,Moderate1
GSM2025745_CTTACCGAGTAAGTAC-1,GSM2025745,8102,2544,4.825969,Moderate,5102,2383,0,0,CD4 T,CD4 T memory,COVID-19,Moderate13
GSM2025743_GGCTGGTAGTTCCACA-1,GSM2025743,2165,1064,10.993072,Moderate,3802,1101,0,0,CD4 T,CD4 T memory,COVID-19,Moderate11


In [58]:
write.table(as.matrix(Moderate@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/Moderate/Moderate_cellphonedb_count.txt', sep='\t', quote=F)


In [59]:
meta_data <- cbind(rownames(Moderate@meta.data), Moderate@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/Moderate/Moderate_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
cellphonedb method statistical_analysis  Moderate_cellphonedb_meta.txt  Moderate_cellphonedb_count.txt      --counts-data=gene_name

## Recover            

In [60]:
Recover<-subset(cellphonedb,subset=Status=="Recover")
table(Recover@meta.data$Type)
Recover@meta.data


       B    CD4 T    CD8 T      cDC Monocyte       NK      pDC Platelet 
    1058     1052      967      676     1077      511      253      799 

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025766_TTGTAGGTCCTCGCAT-1,GSM2025766,1694,815,8.618654,Recover,3880,888,0,0,CD4 T,CD4 T memory,COVID-19,Recover9
GSM2025760_CACACCTGTGCAGGTA-1,GSM2025760,3160,1214,8.544304,Recover,3983,1214,0,0,CD4 T,CD4 T memory,COVID-19,Recover3
GSM2025764_TTGTAGGTCAGTACGT-1,GSM2025764,4691,1627,6.118098,Recover,4528,1627,0,0,CD4 T,CD4 T memory,COVID-19,Recover7
GSM2025767_CGTAGCGGTAGAAAGG-1,GSM2025767,6655,2018,8.054095,Recover,4944,1998,0,0,CD4 T,CD4 T memory,COVID-19,Recover10
GSM2025766_CTACATTTCAGCGATT-1,GSM2025766,4643,1669,4.910618,Recover,4508,1669,0,0,CD4 T,CD4 T memory,COVID-19,Recover9
GSM2025763_ATCACGAAGTGAAGAG-1,GSM2025763,7151,2334,3.579919,Recover,5003,2291,0,0,CD4 T,CD4 T memory,COVID-19,Recover6
GSM2025759_AGCCTAAAGGGTCTCC-1,GSM2025759,4003,1603,4.396702,Recover,4196,1603,0,0,CD4 T,CD4 T memory,COVID-19,Recover2
GSM2025768_CGTAGCGTCAATAAGG-1,GSM2025768,4405,1315,4.245176,Recover,4405,1315,0,0,CD4 T,CD4 T memory,COVID-19,Recover11
GSM2025761_CCCAGTTGTCCAAGTT-1,GSM2025761,3538,1042,4.804975,Recover,4121,1042,9,9,CD4 T,CD4 T memory,COVID-19,Recover4


In [61]:
write.table(as.matrix(Recover@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/Recover/Recover_cellphonedb_count.txt', sep='\t', quote=F)


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.1 GiB”


In [62]:
meta_data <- cbind(rownames(Recover@meta.data), Recover@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/Recover/Recover_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
cellphonedb method statistical_analysis  Recover_cellphonedb_meta.txt  Recover_cellphonedb_count.txt      --counts-data=gene_name

## Severe 

In [63]:
Severe<-subset(cellphonedb,subset=Status=="Severe")
table(Severe@meta.data$Type)
Severe@meta.data


       B    CD4 T    CD8 T      cDC Monocyte       NK      pDC Platelet 
     546      658      663      588      976      828      183      852 

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025752_ACAGCTAAGAGGTTAT-1,GSM2025752,1858,885,11.948332,Severe,3859,951,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025752_GTTCTCGCATCACGTA-1,GSM2025752,4775,1658,5.926702,Severe,4566,1658,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025752_TGACAACTCGCACTCT-1,GSM2025752,3780,1255,4.153439,Severe,4150,1255,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025752_GGCTCGATCATCTGTT-1,GSM2025752,1138,654,15.905097,Severe,3408,860,0,0,CD4 T,CD4 T memory,COVID-19,Severe7
GSM2025750_ACAGCTACAGGTTTCA-1,GSM2025750,3361,1456,3.272835,Severe,3963,1456,0,0,CD4 T,CD4 T memory,COVID-19,Severe5
GSM2025754_AGCTCCTAGGGTCGAT-1,GSM2025754,4736,1628,6.841216,Severe,4545,1628,0,0,CD4 T,CD4 T memory,COVID-19,Severe9
GSM2025753_TGAGAGGCACACGCTG-1,GSM2025753,6416,1734,3.569202,Severe,4861,1723,0,0,CD4 T,CD4 T memory,COVID-19,Severe8
GSM2025748_GCGGGTTAGTGTCCAT-1,GSM2025748,4778,1278,4.771871,Severe,4521,1278,0,0,CD4 T,CD4 T memory,COVID-19,Severe3
GSM2025748_GTGTGCGAGAAACCTA-1,GSM2025748,5653,1745,4.050946,Severe,4790,1745,0,0,CD4 T,CD4 T memory,COVID-19,Severe3


In [64]:
write.table(as.matrix(Severe@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/Severe/Severe_cellphonedb_count.txt', sep='\t', quote=F)


In [65]:
meta_data <- cbind(rownames(Severe@meta.data), Severe@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/Severe/Severe_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
cellphonedb method statistical_analysis  Severe_cellphonedb_meta.txt  Severe_cellphonedb_count.txt      --counts-data=gene_name

## healthy

In [67]:
Healthy<-subset(cellphonedb,subset=Status=="Healthy")
table(Healthy@meta.data$Type)
Healthy@meta.data


       B    CD4 T    CD8 T      cDC Monocyte       NK      pDC Platelet 
     785      820      909      545      682      852      418      449 

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,Status,nCount_SCT,nFeature_SCT,SCT_snn_res.0.1,seurat_clusters,Type,subtype,Status1,orig.ident1
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2025779_GATCTAGAGTTACGGG-1,GSM2025779,4005,1289,3.046192,Healthy,4230,1289,9,9,CD4 T,CD4 T memory,Healthy,Healthy10
GSM2025779_AGCGGTCTCGTGGACC-1,GSM2025779,1634,908,13.157895,Healthy,3753,1010,0,0,CD4 T,CD4 T memory,Healthy,Healthy10
GSM2025772_AGCGTATAGAGGGATA-1,GSM2025772,4950,1886,10.666667,Healthy,4647,1886,0,0,CD4 T,CD4 T memory,Healthy,Healthy3
GSM2025772_GGACGTCGTGATGCCC-1,GSM2025772,2113,724,3.691434,Healthy,4007,756,0,0,CD4 T,CD4 T memory,Healthy,Healthy3
GSM2025777_AGAGTGGGTGTGAAAT-1,GSM2025777,1157,626,14.952463,Healthy,3426,830,0,0,CD4 T,CD4 T memory,Healthy,Healthy8
GSM2025771_GCATGCGTCACTGGGC-1,GSM2025771,6450,2168,4.511628,Healthy,5007,2153,0,0,CD4 T,CD4 T memory,Healthy,Healthy2
GSM2025777_TGCTACCTCTCTGTCG-1,GSM2025777,4672,1461,8.989726,Healthy,4513,1461,0,0,CD4 T,CD4 T memory,Healthy,Healthy8
GSM2025775_CGTCACTCAGTCACTA-1,GSM2025775,5489,1644,4.572782,Healthy,4739,1644,0,0,CD4 T,CD4 T memory,Healthy,Healthy6
GSM2025778_TCACAAGAGAGACGAA-1,GSM2025778,4630,1461,6.436285,Healthy,4496,1461,0,0,CD4 T,CD4 T memory,Healthy,Healthy9


In [68]:
write.table(as.matrix(Healthy@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/healthy/healthy_cellphonedb_count.txt', sep='\t', quote=F)


In [69]:
meta_data <- cbind(rownames(Healthy@meta.data), Healthy@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/status/healthy/healthy_cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
cellphonedb method statistical_analysis  healthy_cellphonedb_meta.txt  healthy_cellphonedb_count.txt      --counts-data=gene_name

In [70]:
c

function (...)  .Primitive("c")

In [ ]:
#导入注释好的seurat对象（已注释）
sce<-readRDS("/data_alluser/CXY/keti/singlecell_eqtl/result/RDS/all_subtype_annotated.rds")

In [ ]:
####筛选部分细胞#####
sample_seob <- function(obj,group.by="Type",sp.size=NULL,diet="true",sp.total=1000) {
all <- obj
if (diet=="true") {
all <- DietSeurat(all)
}
if (is.null(sp.size)) {
nlen <- length(unique(all@meta.data[,group.by]))
sp.size <- ceiling(sp.total/nlen)
}
seob_list <- list()
i <- 1
for (sc in unique(all@meta.data[,group.by])){
cellist <- colnames(all)[which(all@meta.data[,group.by] == sc)]
ob <- subset(all, cells=cellist)
if (length(colnames(ob)) > sp.size) {
ob <- subset(ob,cells=sample(colnames(ob), sp.size))
}
seob_list[[i]] <- ob
i <- i+1
}
all <- Reduce(merge,seob_list)
return(all)
}

sce<- sample_seob(sce,group.by="Type",diet="true",sp.total=30000)

In [ ]:
saveRDS(sce,'/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/sce.rds',compress = F)

In [ ]:
cellphonedb <- readRDS('/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/sce.rds')

In [ ]:
write.table(as.matrix(cellphonedb@assays$RNA@data), '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/cellphonedb_count.txt', sep='\t', quote=F)


In [ ]:
meta_data <- cbind(rownames(cellphonedb@meta.data), cellphonedb@meta.data[,'Type', drop=F])  
meta_data <- as.matrix(meta_data)
meta_data[is.na(meta_data)] = "Unkown" #  细胞类型中不能有NA


In [ ]:
write.table(meta_data, '/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/cellphonedb_meta.txt', sep='\t', quote=F, row.names=F)

In [ ]:
count<-fread("/data_alluser/CXY/keti/singlecell_eqtl/result/cellphonedb/input/cellphonedb_count.txt")